## For 3 categories

In [1]:
import pandas as pd
import numpy as np

In [2]:
data=pd.read_excel("sample tweets.xlsx")

In [3]:
X=data['SentimentText']
Y=data['Sentiment']

In [4]:
with open('../glove.txt',encoding='mbcs') as f:
    doc=f.readlines()
    
glove_emb={
          }
for i in range(len(doc)):
    word   = doc[i].split(" ")[0]
    vector = np.array(doc[i].split(" ")[1:],dtype='float32')
    
    glove_emb[word]=vector

In [5]:
glove_emb['the']

array([ 4.1800e-01,  2.4968e-01, -4.1242e-01,  1.2170e-01,  3.4527e-01,
       -4.4457e-02, -4.9688e-01, -1.7862e-01, -6.6023e-04, -6.5660e-01,
        2.7843e-01, -1.4767e-01, -5.5677e-01,  1.4658e-01, -9.5095e-03,
        1.1658e-02,  1.0204e-01, -1.2792e-01, -8.4430e-01, -1.2181e-01,
       -1.6801e-02, -3.3279e-01, -1.5520e-01, -2.3131e-01, -1.9181e-01,
       -1.8823e+00, -7.6746e-01,  9.9051e-02, -4.2125e-01, -1.9526e-01,
        4.0071e+00, -1.8594e-01, -5.2287e-01, -3.1681e-01,  5.9213e-04,
        7.4449e-03,  1.7778e-01, -1.5897e-01,  1.2041e-02, -5.4223e-02,
       -2.9871e-01, -1.5749e-01, -3.4758e-01, -4.5637e-02, -4.4251e-01,
        1.8785e-01,  2.7849e-03, -1.8411e-01, -1.1514e-01, -7.8581e-01],
      dtype=float32)

In [6]:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
rg=RegexpTokenizer("[a-zA-Z]+")
sw=stopwords.words("english")

In [7]:
unique={
    
}
weight=[]
k=1;
X_train=np.zeros((10000,45))
X_train=X_train.astype('int32')
for i in range(X.shape[0]):
    words=rg.tokenize(X[i])
    words=[w for w in words if w not in sw and w in glove_emb and len(w)>2]
    
    for j in range(min(len(words),45)):
        if words[j].lower() in unique:
            X_train[i][j]=unique[words[j].lower()]
            
        else:
            weight.append(glove_emb[words[j]])
            X_train[i][j]=k
            unique[words[j].lower()]=k
            k+=1

In [8]:
weight_matrix=[np.zeros((50,))]+weight
weight_matrix=np.array(weight_matrix)

In [9]:
weight_matrix.shape

(4185, 50)

In [10]:
from sklearn.preprocessing import LabelEncoder

In [11]:
le=LabelEncoder()

In [12]:
Y_encoded=le.fit_transform(Y)

In [13]:
from keras.utils import to_categorical
Y_train=to_categorical(Y_encoded)

Using TensorFlow backend.


In [14]:
Y_train

array([[1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.]], dtype=float32)

In [15]:
from keras.models import Sequential
from keras.layers import *

In [16]:
model=Sequential()
model.add(Embedding(4185,50, weights=[weight_matrix], input_length=45, trainable=False))
model.add(LSTM(32,return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(32))
model.add(Dropout(0.2))
model.add(Dense(3,activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 45, 50)            209250    
_________________________________________________________________
lstm_1 (LSTM)                (None, 45, 32)            10624     
_________________________________________________________________
dropout_1 (Dropout)          (None, 45, 32)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 99        
Total params: 228,293
Trainable params: 19,043
Non-trainable params: 209,250
___________________________________________

In [17]:
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=['acc'])

In [18]:
hist=model.fit(X_train,Y_train,validation_split=.2,epochs=15,batch_size=32)

Train on 8000 samples, validate on 2000 samples
Epoch 1/15
8000/8000 [==============================] - 10s 1ms/step - loss: 0.7043 - acc: 0.7103 - val_loss: 0.7269 - val_acc: 0.6785
Epoch 2/15
8000/8000 [==============================] - 9s 1ms/step - loss: 0.5452 - acc: 0.7831 - val_loss: 0.6856 - val_acc: 0.7245
Epoch 3/15
8000/8000 [==============================] - 9s 1ms/step - loss: 0.4673 - acc: 0.8227 - val_loss: 0.6556 - val_acc: 0.7545
Epoch 4/15
8000/8000 [==============================] - 9s 1ms/step - loss: 0.4532 - acc: 0.8260 - val_loss: 0.6613 - val_acc: 0.7545
Epoch 5/15
8000/8000 [==============================] - 10s 1ms/step - loss: 0.4067 - acc: 0.8491 - val_loss: 0.6923 - val_acc: 0.7465
Epoch 6/15
8000/8000 [==============================] - 11s 1ms/step - loss: 0.3905 - acc: 0.8515 - val_loss: 0.6080 - val_acc: 0.7480
Epoch 7/15
8000/8000 [==============================] - 10s 1ms/step - loss: 0.3781 - acc: 0.8534 - val_loss: 0.6034 - val_acc: 0.7640
Epoch 8/15

In [23]:
X_train.shape ,Y_train.shape

((10000, 45), (10000, 3))

In [24]:
Y_pred=model.predict_classes(X_train)

In [25]:
from sklearn.metrics import confusion_matrix

In [26]:
confusion_matrix(Y_encoded,Y_pred)

array([[5811,  775,   13],
       [ 324, 2555,   13],
       [ 116,  159,  234]], dtype=int64)

# for 2 categories
- removing entries for neutral

In [27]:
data=pd.read_excel("sample tweets.xlsx")

In [28]:
data2=data[data['Sentiment'] != 'Neutral'].reset_index(drop=True)

In [29]:
X2=data2['SentimentText']
Y2=data2['Sentiment']

In [30]:
unique={
    
}
weight=[]
k=1;
X_train=np.zeros((X2.shape[0],45))
X_train=X_train.astype('int32')
for i in range(X2.shape[0]):
    words=rg.tokenize(X2[i])
    words=[w for w in words if w not in sw and w in glove_emb and len(w)>2]
    
    for j in range(min(len(words),45)):
        if words[j].lower() in unique:
            X_train[i][j]=unique[words[j].lower()]
            
        else:
            weight.append(glove_emb[words[j]])
            X_train[i][j]=k
            unique[words[j].lower()]=k
            k+=1

In [31]:
weight_matrix=[np.zeros((50,))]+weight
weight_matrix=np.array(weight_matrix)

In [32]:
weight_matrix.shape

(3679, 50)

In [33]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
Y_train=le.fit_transform(Y2)

In [34]:
from keras.models import Sequential
from keras.layers import *

In [35]:
model=Sequential()
model.add(Embedding(3679,50, weights=[weight_matrix], input_length=45, trainable=False))
model.add(LSTM(32,return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(32))
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 45, 50)            183950    
_________________________________________________________________
lstm_3 (LSTM)                (None, 45, 32)            10624     
_________________________________________________________________
dropout_3 (Dropout)          (None, 45, 32)            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dropout_4 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 202,927
Trainable params: 18,977
Non-trainable params: 183,950
___________________________________________

In [36]:
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=['acc'])

In [ ]:
hist=model.fit(X_train,Y_train,validation_split=.2,epochs=15,batch_size=32)

Train on 5686 samples, validate on 1422 samples
Epoch 1/15
5686/5686 [==============================] - 7s 1ms/step - loss: 0.2600 - acc: 0.9298 - val_loss: 0.3130 - val_acc: 0.9023
Epoch 2/15
5686/5686 [==============================] - 6s 1ms/step - loss: 0.1738 - acc: 0.9458 - val_loss: 0.3001 - val_acc: 0.8797
Epoch 3/15
5686/5686 [==============================] - 6s 1ms/step - loss: 0.1644 - acc: 0.9502 - val_loss: 0.2688 - val_acc: 0.9015
Epoch 4/15
5686/5686 [==============================] - 6s 1ms/step - loss: 0.1470 - acc: 0.9590 - val_loss: 0.2933 - val_acc: 0.9121
Epoch 5/15
5686/5686 [==============================] - 6s 1ms/step - loss: 0.1388 - acc: 0.9610 - val_loss: 0.2757 - val_acc: 0.9100
Epoch 6/15
5686/5686 [==============================] - 7s 1ms/step - loss: 0.1464 - acc: 0.9590 - val_loss: 0.2811 - val_acc: 0.9100
Epoch 7/15
5686/5686 [==============================] - 7s 1ms/step - loss: 0.1314 - acc: 0.9620 - val_loss: 0.2650 - val_acc: 0.9191
Epoch 8/15
568

In [75]:
Y_pred=model.predict_classes(X_train)

In [76]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_train,Y_pred)

array([[6580,   19],
       [ 181,  328]], dtype=int64)